# FDTD Sparameters in Meep

Here are some examples on how to extract Sparameters in Meep.

For extracting Sparameters, `gmeep` automatically moves the source between ports.

- add monitors
- run simulation
- pull coefficients, and create proper ratios to get Sparameters. Monitors are recording Fourier Transform fields. Sparameter is a relationship of those parameters. Frequency domain approach at many different frequencies. Get eigenmode coefficients.
    - forward coefficient: how much waveguide forward mode
    - backward coefficient: how much waveguide forward mode

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import gdsfactory as gf
import gdsfactory.simulation.gmeep as gm

gf.config.set_plot_options(show_subports=False, show_ports=False)

In [ ]:
c = gf.components.straight(length=2)
p = 2
c = gf.add_padding_container(c, default=0, top=p, bottom=p)
c

In [ ]:
gm.get_sparametersNxN(c, run=False)

In [ ]:
df = gm.get_sparametersNxN(c)
gf.simulation.plot.plot_sparameters(df)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=('s21m',))

It's a bit strange that we get some gain in a passive waveguide. S21 should be around 0dB (100% transmission)

Lets increase the resolution.

In [ ]:
df = gm.get_sparametersNxN(c, resolution=60)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=('s21m',))

In [ ]:
c = gf.components.straight(length=2)
p = 2
c = gf.add_padding_container(c, default=0, top=p, bottom=p)
df = gm.get_sparametersNxN(c, resolution=80)
gf.simulation.plot.plot_sparameters(df, keys=('s21m',))

In [ ]:
component_name = 'mmi1x2'
c = gf.components.factory[component_name]()
p = 3
cm = gf.add_padding_container(c, default=0, top=p, bottom=p)
cm

In [ ]:
df = gm.get_sparametersNxN(c, resolution=60)
gf.simulation.plot.plot_sparameters(df)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=['s12m'])

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=['s13m'])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import gdsfactory as gf
import gdsfactory.simulation.gmeep as gm

component_name = 'coupler_ring'
c = gf.components.factory[component_name]()
p = 3
c = gf.add_padding_container(c, default=0, left=p, right=p, bottom=p)
df = gm.get_sparametersNxN(c, run=False)

In [ ]:
df = gm.get_sparametersNxN(c)
gf.simulation.plot.plot_sparameters(df)